In [ ]:
%%capture
!pip install transformers

In [ ]:
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel

In [ ]:
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [ ]:
# Each input text should start with "query: " or "passage: ".
# For tasks other than retrieval, you can simply use the "query: " prefix.
input_texts = ['query: how much protein should a female eat',
               'query: summit define',
               "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
               "passage: Definition of summit for English Language Learners. : 1  the highest point of a mountain : the top of a mountain. : 2  the highest level. : 3  a meeting or series of meetings between the leaders of two or more governments."]


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('intfloat/e5-large-v2')

In [ ]:
model = AutoModel.from_pretrained('intfloat/e5-large-v2')

In [ ]:
# Tokenize the input texts
# a batch of input text refers to a set of input data that is processed together as a group
batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

outputs = model(**batch_dict)
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

In [ ]:
print(batch_dict)

{'input_ids': tensor([[  101, 23032,  1024,  2129,  2172,  5250,  2323,  1037,  2931,  4521,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0],
        [  101, 23032,  1024,  6465,  9375,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0, 

In [ ]:
print(embeddings)

tensor([[ 0.1340, -1.4772,  0.5544,  ..., -1.3651,  0.0587, -0.1317],
        [ 0.0170, -0.8781, -0.1351,  ..., -0.9939,  0.0988,  0.7327],
        [ 0.2937, -0.9741,  0.5673,  ..., -1.0390,  0.1233, -0.0732],
        [ 0.3453, -0.4452, -0.0427,  ..., -0.5100,  0.4413,  0.5360]],
       grad_fn=<DivBackward0>)


In [ ]:
print(outputs)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-1.0530e-01, -1.1656e+00,  5.2558e-01,  ..., -1.3029e+00,
           7.5668e-02,  1.2698e-01],
         [ 1.6324e-01, -1.2218e+00,  8.8644e-01,  ..., -1.2399e+00,
          -6.6569e-02, -1.4282e-01],
         [ 2.7411e-01, -1.7795e+00,  6.6177e-01,  ..., -1.2811e+00,
          -1.5314e-02, -3.0681e-01],
         ...,
         [ 2.2556e-01, -1.1561e+00,  1.6384e+00,  ..., -1.0209e+00,
           1.2710e-01, -1.2909e-01],
         [ 2.3378e-01, -1.1948e+00,  1.7149e+00,  ..., -1.0953e+00,
           1.4249e-01, -2.1865e-01],
         [ 2.5497e-01, -1.2201e+00,  1.6000e+00,  ..., -1.1600e+00,
           1.7370e-01, -1.4113e-01]],

        [[ 2.3788e-02, -9.0967e-01, -4.3280e-01,  ..., -1.2103e+00,
           1.1593e-01,  7.0294e-01],
         [ 1.2813e-03, -7.0803e-01,  1.9633e-02,  ..., -8.7828e-01,
           1.0220e-01,  7.0700e-01],
         [-5.2274e-02, -9.8783e-01,  5.0283e-03,  ..., -8.3544e-01,
          -4.

In [ ]:
# (Optionally) normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
scores = (embeddings[:2] @ embeddings[2:].T) * 100
print(scores.tolist())